In [2]:
import pandas as pd
import numpy as np
import math
import nltk
from string import punctuation
from nltk.corpus import stopwords

In [3]:
df = pd.read_csv("amazon_alexa.tsv", sep = "\t")
#df.drop("Unnamed: 0", axis = 1, inplace = True) se si importa il test_set
df

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1
...,...,...,...,...,...
3145,5,30-Jul-18,Black Dot,"Perfect for kids, adults and everyone in betwe...",1
3146,5,30-Jul-18,Black Dot,"Listening to music, searching locations, check...",1
3147,5,30-Jul-18,Black Dot,"I do love these things, i have them running my...",1
3148,5,30-Jul-18,White Dot,Only complaint I have is that the sound qualit...,1


In [4]:
data_dict = {1:{"tokens":[], "text": "", "freq":{}, "sents":[]}, 0:{"tokens":[], "text": "", "freq":{}, "sents":[]}, "total_text":{"tokens":[], "text": "", "freq":{}, "sents":[]}}

for review in df["verified_reviews"]:
    data_dict["total_text"]["text"] += " " + str(review)

for review in df[df["feedback"] == 0]["verified_reviews"]:
    data_dict[0]["text"] += " " + str(review)

for review in df[df["feedback"] == 1]["verified_reviews"]:
    data_dict[1]["text"] += " " + str(review)


stops = stopwords.words('english')
tk = nltk.tokenize.TweetTokenizer()
lemmatizer = nltk.WordNetLemmatizer()
for k in data_dict:
    text = data_dict[k]["text"]
    sents = nltk.tokenize.sent_tokenize(text, "english")

    tokens_per_sents = []
    total_tokens = []
    for sent in sents:
        tokens = tk.tokenize(sent)
        lower_tokens = [w.lower() for w in tokens]
        clean_tokens = [lemmatizer.lemmatize(w) for w in lower_tokens if w not in punctuation and w not in stops and len(w)>2]
        tokens_per_sents.append(clean_tokens)
        total_tokens.extend(clean_tokens)

    data_dict[k]["tokens"] = total_tokens
    data_dict[k]["sents"] = tokens_per_sents
    freq = nltk.FreqDist(total_tokens)
    data_dict[k]["freq"] = freq

    total_bigrams = []
    for sent in tokens_per_sents:
        bigrams = list(nltk.ngrams(sent, 2))
        total_bigrams.extend(bigrams)

    data_dict[k]["bigrams"] = total_bigrams
    data_dict[k]["bigrams_freq"] = nltk.FreqDist(total_bigrams)


In [5]:
data_dict[0]["bigrams_freq"].most_common()

[(('echo', 'dot'), 32),
 (('thing', 'try'), 11),
 (('echo', 'plus'), 10),
 (('play', 'music'), 9),
 (('echo', 'show'), 9),
 (('sound', 'quality'), 8),
 (('work', 'fine'), 7),
 (('prime', 'day'), 6),
 (('stopped', 'working'), 6),
 (('amazon', 'would'), 6),
 (("i'm", 'sure'), 6),
 (('google', 'home'), 6),
 (('half', 'time'), 6),
 (('need', 'prime'), 6),
 (('sound', 'terrible'), 5),
 (('echo', 'device'), 5),
 (('much', 'better'), 5),
 (('customer', 'service'), 5),
 (('really', 'disappointed'), 4),
 (('waste', 'money'), 4),
 (('would', 'recommend'), 4),
 (('buy', 'refurbished'), 4),
 (('refurbished', 'echo'), 4),
 (('great', 'product'), 4),
 (('never', 'buy'), 4),
 (('certified', 'refurbished'), 4),
 (('...', 'even'), 4),
 (('buy', 'another'), 4),
 (('sound', 'like'), 4),
 (('turn', 'thing'), 4),
 (('try', 'screen'), 4),
 (('echo', 'spot'), 4),
 (('stop', 'working'), 4),
 (('bluetooth', 'speaker'), 4),
 (('light', 'bulb'), 4),
 (('hue', 'bridge'), 4),
 (('fire', 'stick'), 4),
 (('ask', 'pl

In [54]:
word_scores = {1:{}, 0:{}}
bigram_scores = {1:{}, 0:{}}

for k in data_dict:
    if k != "total_text":

        for w in set(data_dict[k]["tokens"]):
            p = data_dict[k]["freq"][w]/len(data_dict[k]["tokens"])
            p_category = data_dict[k]["freq"][w]/data_dict["total_text"]["freq"][w]
            h = -math.log2(p)
            score = h * p_category * p
            word_scores[k][w] = score

        for bigram in set(data_dict[k]["bigrams"]):
            p = data_dict[k]["bigrams_freq"][bigram]/len(data_dict[k]["bigrams"])
            p_category = data_dict[k]["bigrams_freq"][bigram]/data_dict["total_text"]["bigrams_freq"].get(bigram, 1)
            h = -math.log2(p)
            score = p_category * p
            bigram_scores[k][bigram] = score

In [60]:
for label in word_scores:
    values_list = list(word_scores[label].values())
    min_values = min(values_list)
    max_values = max(values_list)
    for k in word_scores[label]:
        val = word_scores[label][k]
        word_scores[label][k] = (val- min_values)/(max_values-min_values)

In [61]:
for label in bigram_scores:
    values_list = list(bigram_scores[label].values())
    min_values = min(values_list)
    max_values = max(values_list)
    for k in bigram_scores[label]:
        val = bigram_scores[label][k]
        bigram_scores[label][k] = (val- min_values)/(max_values-min_values)

In [62]:
{k: v for k, v in sorted(bigram_scores[1].items(), key=lambda item: item[1], reverse=True)}

{('work', 'great'): 1.0,
 ('echo', 'dot'): 0.9466305445952701,
 ('easy', 'set'): 0.8909674410949887,
 ('love', 'echo'): 0.7345421285611098,
 ('sound', 'quality'): 0.5481129976562952,
 ('prime', 'day'): 0.5462373182939759,
 ('easy', 'use'): 0.5367602015159412,
 ('play', 'music'): 0.535695876877783,
 ('still', 'learning'): 0.5027018130948788,
 ('great', 'product'): 0.4911551887277382,
 ('echo', 'plus'): 0.4494104523889814,
 ('love', 'alexa'): 0.42777335856854193,
 ('echo', 'show'): 0.3875476403523143,
 ('great', 'sound'): 0.38054572662466896,
 ('love', 'love'): 0.3802177569583921,
 ('work', 'well'): 0.36749001106326173,
 ('fire', 'stick'): 0.28690786405902846,
 ('echo', 'spot'): 0.28690786405902846,
 ('alarm', 'clock'): 0.2780748606248032,
 ('smart', 'home'): 0.2724374913785667,
 ('playing', 'music'): 0.27126695463366063,
 ('listen', 'music'): 0.27126695463366063,
 ('turn', 'light'): 0.23095172442998216,
 ('easy', 'setup'): 0.22342302804216843,
 ('really', 'like'): 0.21736820343397956,
 

In [56]:
test_set = pd.read_csv("test_set.csv", sep = "\t")
test_set.drop("Unnamed: 0", axis = 1, inplace = True)
test_set

,variation,verified_reviews,rating,date,feedback
0,Echo 4th Gen,I wish we never purchased this I hate the alex...,1,"October 25, 2022",0
1,Echo 4th Gen,"Cons: it cannot Cast audio from my phone, does...",2,"October 25, 2022",0
2,Echo 4th Gen,Love this Echo. It is bigger than expected but...,4,"October 25, 2022",1
3,Echo 4th Gen,First off I would give this premium sound echo...,4,"October 25, 2022",1
4,Echo 4th Gen,Hubiera dado mejor puntuación si hubiera podid...,4,"October 24, 2022",1
...,...,...,...,...,...
1465,echo show 10,Product seems fine. Dislike the unexpected ads...,4,"August 4, 2022",1
1466,echo show 10,I really like it.,5,"August 4, 2022",1
1467,echo show 10,Roomie got this 'thing' from a friend as a gif...,1,"August 4, 2022",0
1468,echo show 10,Excelente producto a la medida. Lo recomiendo ...,5,"August 4, 2022",1


In [68]:
from collections import Counter

output = []
for review in test_set[test_set["feedback"] == 1]["verified_reviews"]:
    tokens = tk.tokenize(str(review))
    lower_tokens = [w.lower() for w in tokens]
    clean_tokens = [lemmatizer.lemmatize(w) for w in lower_tokens if w not in punctuation and w not in stops and len(w)>2]
    freq = nltk.FreqDist(clean_tokens)
    score = {0:0, 1:0}
    for k in [0, 1]:
        for w in clean_tokens:
            val = word_scores[k].get(w, 0)
            #h = - math.log2(freq[w]/len(clean_tokens))
            score[k]+=val
    output.append(max(score, key=score.get))

result = Counter(output)
total_samples = sum(result.values())
accuracy = result[1]/total_samples
print(f"Accuracy for {1}:", accuracy)


Accuracy for 1: 0.8007843137254902


In [71]:
output = []
for review in test_set[test_set["feedback"] == 0]["verified_reviews"]:
    tokens = tk.tokenize(str(review))
    lower_tokens = [w.lower() for w in tokens]
    clean_tokens = [lemmatizer.lemmatize(w) for w in lower_tokens if w not in punctuation and w not in stops and len(w)>2]
    freq = nltk.FreqDist(clean_tokens)
    bigrams = list(nltk.ngrams(clean_tokens, 2))
    bigrams_freq = nltk.FreqDist(bigrams)
    score = {0:0, 1:0}

    for k in [0, 1]:
        for bi in bigrams:
            val = bigram_scores[k].get(bi, 0)
            #h = - math.log2(bigrams_freq[bi]/len(bigrams))
            score[k]+=val

    for k in [0, 1]:
        for w in clean_tokens:
            val = word_scores[k].get(w, 0)
            #h = - math.log2(freq[w]/len(clean_tokens))
            score[k]+=val

    output.append(max(score, key=score.get))

Counter(output)

result = Counter(output)
total_samples = sum(result.values())
accuracy = result[0]/total_samples
print(f"Accuracy for {0}:", accuracy)

Accuracy for 0: 0.676923076923077
